In [1]:
import pandas as pd
import numpy as np

In [2]:
# fs is the whole original schedule
fs = pd.read_csv("flight_schedule.csv", sep=',')

In [3]:
fs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2364 entries, 0 to 2363
Data columns (total 14 columns):
flight_id                 2364 non-null int64
date                      2364 non-null object
domistic/international    2364 non-null object
flight_num                2364 non-null int64
de_ap                     2364 non-null int64
la_ap                     2364 non-null int64
de_time                   2364 non-null object
la_time                   2364 non-null object
aircraft_id               2364 non-null int64
aircraft_type             2364 non-null int64
passenger_num             2364 non-null int64
connect_passenger_num     2364 non-null int64
seat_num                  2364 non-null int64
important_factor          2364 non-null float64
dtypes: float64(1), int64(9), object(4)
memory usage: 258.6+ KB


In [4]:
# make copy of fs
fs_copy = fs.copy()

In [5]:
#change the type of departure time and landing time and date
fs_copy.de_time = pd.to_datetime(fs_copy.de_time)
fs_copy.la_time = pd.to_datetime(fs_copy.la_time)
fs_copy.date = pd.to_datetime(fs_copy.date)

In [6]:
# extract domestic flights on 06/05/2017
fs_0506 = fs_copy[fs_copy.date == pd.Timestamp(year=2017,month=5, day=6)][fs_copy['domistic/international'] == 'domestic']

/home/big_muscle/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [7]:
len(fs_0506.flight_num.unique())

337

In [8]:
# flight id is unrepeatable
len(fs_0506.flight_id.unique())

461

In [9]:
# 创建表格?
#_test_dataset = pd.DataFrame()

### Define 1

- change 'flight_id' and 'aircraft_id' type, add "a" before aircraft id, add "f" before flight id.
- set flight_id as index

### Code

In [10]:
# duplicate fs_0506
fs_0506_copy = fs_0506.copy()

In [11]:
fs_0506_copy[['flight_id']]=fs_0506_copy[['flight_id']].astype(str)
fs_0506_copy[['aircraft_id']]=fs_0506_copy[['aircraft_id']].astype(str)

In [12]:
fs_0506_copy['flight_id']='f'+fs_0506_copy['flight_id']
fs_0506_copy['aircraft_id']='a'+fs_0506_copy['aircraft_id']

In [13]:
fs_0506_copy = fs_0506_copy.set_index('flight_id')

### Test

In [14]:
fs_0506_copy.head()

,date,domistic/international,flight_num,de_ap,la_ap,de_time,la_time,aircraft_id,aircraft_type,passenger_num,connect_passenger_num,seat_num,important_factor
flight_id,,,,,,,,,,,,,
f10,2017-05-06,domestic,345,50,48,2017-05-06 08:05:00,2017-05-06 11:00:00,a40,1,100,0,119,1.0
f6,2017-05-06,domestic,344,68,49,2017-05-06 11:15:00,2017-05-06 13:25:00,a6,2,90,0,161,1.0
f16,2017-05-06,domestic,349,50,72,2017-05-06 07:30:00,2017-05-06 10:25:00,a94,2,131,0,161,1.0
f25,2017-05-06,domestic,356,72,49,2017-05-06 15:15:00,2017-05-06 17:55:00,a29,3,103,0,163,1.0
f27,2017-05-06,domestic,364,50,72,2017-05-06 16:25:00,2017-05-06 19:25:00,a101,2,92,0,161,1.0


### Function 1
get the path between flight1 and flight2

In [15]:
# the path between flight1 and flight2
def ff_connect(dataset, f_id):
    de_ap1, de_time1 = dataset.loc[f_id, ['de_ap', 'de_time']] #提取对应ID的departure airpor和departure time
    
    value = dataset.loc[dataset['la_ap']==de_ap1].loc[dataset['de_time'] < de_time1]
    return value.index

print(type(ff_connect(fs_0506_copy,'f27')))

<class 'pandas.core.indexes.base.Index'>


### Define 2
establish flight to flight phromone matrix 'ffp1'

### Code

In [16]:
# extract 0506 flight_id
f_index1 = fs_0506_copy.index.values

In [17]:
# flight to flight pheromone matrix
ffp1 = pd.DataFrame(data=np.zeros((461, 461)), index=f_index1, columns=f_index1)

In [18]:
for index in f_index1:
    ffp1.loc[ff_connect(fs_0506_copy,index), index] = 1

### Test
(id1,id2), the path is from id1 to id2

In [19]:
f_index1

array(['f10', 'f6', 'f16', 'f25', 'f27', 'f35', 'f39', 'f48', 'f50', 'f53',
       'f57', 'f59', 'f63', 'f994', 'f74', 'f77', 'f80', 'f87', 'f98',
       'f101', 'f113', 'f123', 'f127', 'f130', 'f136', 'f139', 'f141',
       'f147', 'f155', 'f162', 'f174', 'f184', 'f187', 'f191', 'f199',
       'f203', 'f217', 'f225', 'f231', 'f240', 'f250', 'f252', 'f257',
       'f259', 'f270', 'f273', 'f278', 'f282', 'f302', 'f311', 'f319',
       'f335', 'f339', 'f341', 'f1127', 'f367', 'f368', 'f375', 'f376',
       'f382', 'f387', 'f388', 'f392', 'f394', 'f397', 'f400', 'f401',
       'f403', 'f407', 'f411', 'f415', 'f420', 'f431', 'f441', 'f445',
       'f453', 'f458', 'f464', 'f470', 'f473', 'f474', 'f488', 'f492',
       'f496', 'f499', 'f501', 'f502', 'f512', 'f517', 'f518', 'f520',
       'f523', 'f524', 'f526', 'f531', 'f533', 'f536', 'f550', 'f553',
       'f554', 'f559', 'f564', 'f569', 'f571', 'f578', 'f586', 'f593',
       'f597', 'f1000', 'f613', 'f624', 'f631', 'f635', 'f644', 'f648',

In [20]:
ffp1

,f10,f6,f16,f25,f27,f35,f39,f48,f50,f53,...,f2325,f2332,f2335,f2336,f2346,f2347,f2348,f2349,f2359,f2363
f10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f16,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f27,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
f50,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Define 3
- establish a dataframe to save the initial status of each aircraft
- include aircraft_id(index), de_ap(initial airport)， seat_num

### Code

In [21]:
# the unique aircraft id on 0506
a_index = fs_0506_copy.aircraft_id.unique()

#### Test1

In [22]:
a_index

array(['a40', 'a6', 'a94', 'a29', 'a101', 'a12', 'a7', 'a134', 'a45',
       'a121', 'a5', 'a77', 'a131', 'a33', 'a141', 'a10', 'a15', 'a90',
       'a91', 'a19', 'a41', 'a30', 'a65', 'a48', 'a44', 'a73', 'a53',
       'a68', 'a69', 'a117', 'a58', 'a43', 'a92', 'a76', 'a108', 'a74',
       'a133', 'a132', 'a126', 'a128', 'a50', 'a22', 'a21', 'a35', 'a25',
       'a28', 'a20', 'a63', 'a27', 'a26', 'a84', 'a88', 'a109', 'a81',
       'a9', 'a106', 'a71', 'a98', 'a2', 'a47', 'a46', 'a130', 'a42',
       'a142', 'a124', 'a59', 'a8', 'a103', 'a36', 'a93', 'a64', 'a135',
       'a54', 'a127', 'a140', 'a96', 'a18', 'a31', 'a119', 'a56', 'a49',
       'a114', 'a87', 'a78', 'a70', 'a52', 'a55', 'a67', 'a37', 'a38',
       'a137', 'a72', 'a79', 'a39', 'a83', 'a62', 'a14', 'a102', 'a61',
       'a51', 'a129', 'a80', 'a100', 'a139', 'a34', 'a4', 'a60', 'a32',
       'a86', 'a82', 'a116', 'a110', 'a75', 'a123', 'a125', 'a136', 'a120',
       'a112', 'a113', 'a66', 'a115'], dtype=object)

In [23]:
len(a_index)

121

### group by aircraft_id and extract the min 'de_time' of each aircraft

In [24]:
earliest_de_time = fs_0506_copy.groupby(['aircraft_id'])['de_time'].min().reset_index() 
# reset index 为了把series 变成df

#### Test2

In [25]:
earliest_de_time.head()

,aircraft_id,de_time
0,a10,2017-05-06 09:20:00
1,a100,2017-05-06 07:10:00
2,a101,2017-05-06 08:50:00
3,a102,2017-05-06 17:45:00
4,a103,2017-05-06 08:35:00


### 只保留fs_0506_copy中de_time和aircraft_id与earliest_de_time表中相同的部分，并取其中aircraft_id, de_ap, seat_num

In [26]:
# merge 默认是inner联结
a_status = fs_0506_copy.merge(earliest_de_time, on=['de_time', 'aircraft_id'])[['aircraft_id', 'de_ap', 'seat_num']]

In [27]:
# set aircraft_id as index of a_status
a_status=a_status.set_index('aircraft_id')

### Test

In [28]:
a_status.head()

,de_ap,seat_num
aircraft_id,,
a40,50,119
a94,50,161
a12,50,161
a29,49,163
a45,49,119


### Function 2
define the path from aircraft to flight

In [29]:
def af_connect(a_id, dataset1, dataset2):
    de_ap1, seat_num1 = dataset1.loc[a_id, ['de_ap', 'seat_num']]
    value = dataset2.loc[dataset2['de_ap']==de_ap1].loc[dataset2['passenger_num'] <= seat_num1]
    
    return value.index

# test
print(af_connect('a90',a_status,fs_0506_copy))

Index(['f59', 'f74', 'f77', 'f136', 'f147', 'f155', 'f162', 'f174', 'f199',
       'f203', 'f231', 'f252', 'f282', 'f335', 'f394', 'f445', 'f496', 'f501',
       'f526', 'f531', 'f536', 'f559', 'f597', 'f655', 'f742', 'f1242',
       'f1012', 'f43', 'f437', 'f669', 'f1115', 'f166', 'f1087', 'f765',
       'f1558', 'f871', 'f1809', 'f677', 'f814', 'f891', 'f863', 'f1163',
       'f848', 'f1044', 'f1205', 'f2103', 'f2262', 'f476', 'f1867', 'f2214',
       'f242', 'f700', 'f776', 'f1040', 'f484', 'f934', 'f2301', 'f1148',
       'f1853', 'f1035', 'f480', 'f178', 'f1228', 'f1275', 'f1296', 'f1650',
       'f2116', 'f2346'],
      dtype='object', name='flight_id')


### Define 4
- establish aircraft to flight pheromone matrix
- the path from aircraft to flight

### Code

In [30]:
# aircraft to flight pheromone matrix
afp1 = pd.DataFrame(data=np.zeros((121, 461)), index=a_index, columns=f_index1)

In [31]:
for a_id in a_index:
    afp1.loc[a_id,af_connect(a_id, a_status,fs_0506_copy)] = 1

### Test

In [32]:
afp1

,f10,f6,f16,f25,f27,f35,f39,f48,f50,f53,...,f2325,f2332,f2335,f2336,f2346,f2347,f2348,f2349,f2359,f2363
a40,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
a6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
a94,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
a29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
a101,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
a12,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
a7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
a134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
a45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
a121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Define 5
flight to aircraft pheromone matrix

### Code

In [33]:
# flight to aircraft pheromone matrix
fap1 = pd.DataFrame(data=np.ones((461, 121)), index=f_index1, columns=a_index)

### Test

In [34]:
fap1

,a40,a6,a94,a29,a101,a12,a7,a134,a45,a121,...,a110,a75,a123,a125,a136,a120,a112,a113,a66,a115
f10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f16,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f25,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f27,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f35,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f39,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f48,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f50,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f53,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


### Define 6
aircraft to aircraft pheromone matrix

### Code

In [35]:
# aircrfat to aircraft pheromone matrix
aap1 = pd.DataFrame(data=np.ones((121, 121)), index=a_index, columns=a_index)

In [36]:
for a_id in a_index:
    aap1.loc[a_id,a_id] = 0

### Test

In [37]:
aap1

,a40,a6,a94,a29,a101,a12,a7,a134,a45,a121,...,a110,a75,a123,a125,a136,a120,a112,a113,a66,a115
a40,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
a6,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
a94,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
a29,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
a101,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
a12,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
a7,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
a134,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
a45,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
a121,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


### Define 7
merge aap1, afp1, fap1, ffp1

### code

In [38]:
# concat轴相连接（afp1+ffp1）,（aap1+fap1）
Ap1 = pd.concat([aap1,fap1])
Fp1 = pd.concat([afp1,ffp1])

#### Test1

In [39]:
#Fp1

In [40]:
#Ap1

In [41]:
initial_p1 = Ap1.join(Fp1)

In [44]:
initial_p1.to_csv("initial_p1.csv")

In [43]:
fs_0506_copy.to_csv('fs0506.csv')